# Create a Forecast using Target, Related and Item-meta Data
* **About 40 mins may be elapsed**

In [1]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [2]:
%store -r

In [3]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Create Prophet and DeepAR+ Campaign

In [4]:
# Prophet
item_meta_prophet_forecastName = project+'_prophet_algo_forecast' + item_meta_suffix + suffix
prophet_create_forecast_response=forecast.create_forecast(
    ForecastName = item_meta_prophet_forecastName,
    PredictorArn = item_meta_prophet_predictor_arn)


In [5]:
item_meta_prophet_forecast_arn = prophet_create_forecast_response['ForecastArn']
forecast.describe_forecast(ForecastArn = item_meta_prophet_forecast_arn)

{'ForecastArn': 'arn:aws:forecast:us-east-2:057716757052:forecast/WalmartKaggleWithThreeDatasets_prophet_algo_forecast_ItemM92608',
 'ForecastName': 'WalmartKaggleWithThreeDatasets_prophet_algo_forecast_ItemM92608',
 'ForecastTypes': ['0.1', '0.5', '0.9'],
 'PredictorArn': 'arn:aws:forecast:us-east-2:057716757052:predictor/WalmartKaggleWithThreeDatasets_prophet_algo_1_ItemM92608',
 'DatasetGroupArn': 'arn:aws:forecast:us-east-2:057716757052:dataset-group/WalmartKaggleWithThreeDatasetsDSG_ItemM35988',
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 4, 5, 7, 18, 33, 709000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 5, 7, 18, 33, 709000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'b0f00310-dbb3-4184-bb6b-8dc47a39ed5a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 07:18:34 GMT',
   'x-amzn-requestid': 'b0f00310-dbb3-4184-bb6b-8dc47a39ed5a',
   'content-leng

In [6]:
# DeepAR+
item_meta_deeparp_forecastName = project+'_deeparp_algo_forecast' + item_meta_suffix + suffix
deeparp_create_forecast_response=forecast.create_forecast(
    ForecastName = item_meta_deeparp_forecastName,
    PredictorArn = item_meta_deepar_predictor_arn)


In [7]:
item_meta_deeparp_forecast_arn = deeparp_create_forecast_response['ForecastArn']
forecast.describe_forecast(ForecastArn = item_meta_deeparp_forecast_arn)

{'ForecastArn': 'arn:aws:forecast:us-east-2:057716757052:forecast/WalmartKaggleWithThreeDatasets_deeparp_algo_forecast_ItemM92608',
 'ForecastName': 'WalmartKaggleWithThreeDatasets_deeparp_algo_forecast_ItemM92608',
 'ForecastTypes': ['0.1', '0.5', '0.9'],
 'PredictorArn': 'arn:aws:forecast:us-east-2:057716757052:predictor/WalmartKaggleWithThreeDatasets_deeparp_algo_1_ItemM92608',
 'DatasetGroupArn': 'arn:aws:forecast:us-east-2:057716757052:dataset-group/WalmartKaggleWithThreeDatasetsDSG_ItemM35988',
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 4, 5, 7, 18, 34, 694000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 5, 7, 18, 34, 694000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'a130531b-94dd-4bd7-88c3-8b11f8109f77',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 07:18:35 GMT',
   'x-amzn-requestid': 'a130531b-94dd-4bd7-88c3-8b11f8109f77',
   'content-leng

In [8]:
%%time
# Check the Prophet status

while True:
    createProphetStatus = forecast.describe_forecast(ForecastArn= item_meta_prophet_forecast_arn)['Status']
    createDeeparpStatus = forecast.describe_forecast(ForecastArn= item_meta_deeparp_forecast_arn)['Status']    
    print("Prophet: ", createProphetStatus)
    print("DeepARP: ", createProphetStatus)          
    if createProphetStatus != 'ACTIVE' and createProphetStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpStatus != 'ACTIVE' and createDeeparpStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_PENDING
DeepARP:  CREATE_PENDING
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PR

## Upload forecast results to S3

In [9]:
meta_data_prophet_path = "s3://" + bucket_name + "/" + bucket_folder + "/prophet_" + item_meta_suffix + suffix + "/" 
meta_data_prophet_job_name = "ProphetExport1" + item_meta_suffix + suffix
create_forecast_export_job_prophet_response = forecast.create_forecast_export_job(
    ForecastExportJobName = meta_data_prophet_job_name,
    ForecastArn = item_meta_prophet_forecast_arn,
    Destination={
        "S3Config" : {
            "Path": meta_data_prophet_path,
            "RoleArn": role_arn
        }
    })


In [10]:
ItemMetaForecastProphetExportJobArn = create_forecast_export_job_prophet_response["ForecastExportJobArn"]
forecast.describe_forecast_export_job(ForecastExportJobArn = ItemMetaForecastProphetExportJobArn)

{'ForecastExportJobArn': 'arn:aws:forecast:us-east-2:057716757052:forecast-export-job/WalmartKaggleWithThreeDatasets_prophet_algo_forecast_ItemM92608/ProphetExport1_ItemM92608',
 'ForecastExportJobName': 'ProphetExport1_ItemM92608',
 'ForecastArn': 'arn:aws:forecast:us-east-2:057716757052:forecast/WalmartKaggleWithThreeDatasets_prophet_algo_forecast_ItemM92608',
 'Destination': {'S3Config': {'Path': 's3://sagemaker-us-east-2-057716757052/WalmartKaggleWithThreeDatasets/prophet__ItemM92608',
   'RoleArn': 'arn:aws:iam::057716757052:role/ForecastRoleWalmart35988'}},
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 4, 5, 7, 47, 40, 777000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 5, 7, 47, 40, 777000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'e7f417d0-256f-418b-861d-a31e62607b86',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 07:47:40 GMT',
   'x-amzn-requ

In [11]:
item_meta_deeparp_path = "s3://" + bucket_name + "/" + bucket_folder  + "/deeparp_" + item_meta_suffix + suffix + "/" 
item_meta_deeparp_job_name = "DeepARPExport1" + item_meta_suffix + suffix
create_forecast_export_job_deepar_response = forecast.create_forecast_export_job(
    ForecastExportJobName=item_meta_deeparp_job_name,
    ForecastArn=item_meta_deeparp_forecast_arn,
    Destination={
        "S3Config" : {
            "Path": item_meta_deeparp_path,
            "RoleArn": role_arn
        }
    })

In [12]:
ItemMetaForecastDeeparExportJobArn = create_forecast_export_job_deepar_response["ForecastExportJobArn"]
forecast.describe_forecast_export_job(ForecastExportJobArn = ItemMetaForecastDeeparExportJobArn)

{'ForecastExportJobArn': 'arn:aws:forecast:us-east-2:057716757052:forecast-export-job/WalmartKaggleWithThreeDatasets_deeparp_algo_forecast_ItemM92608/DeepARPExport1_ItemM92608',
 'ForecastExportJobName': 'DeepARPExport1_ItemM92608',
 'ForecastArn': 'arn:aws:forecast:us-east-2:057716757052:forecast/WalmartKaggleWithThreeDatasets_deeparp_algo_forecast_ItemM92608',
 'Destination': {'S3Config': {'Path': 's3://sagemaker-us-east-2-057716757052/WalmartKaggleWithThreeDatasets/deeparp__ItemM92608',
   'RoleArn': 'arn:aws:iam::057716757052:role/ForecastRoleWalmart35988'}},
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 4, 5, 7, 47, 41, 53000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 5, 7, 47, 41, 53000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '86725499-fb19-48d8-b342-3914045dff05',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 07:47:41 GMT',
   'x-amzn-reques

In [13]:
%%time
# Check the Prophet status

while True:
    createProphetStatus = forecast.describe_forecast_export_job(ForecastExportJobArn= ItemMetaForecastProphetExportJobArn)['Status']
    createDeeparpStatus = forecast.describe_forecast_export_job(ForecastExportJobArn= ItemMetaForecastDeeparExportJobArn)['Status']    
    print("Prophet: ", createProphetStatus)
    print("DeepARP: ", createProphetStatus)          
    if createProphetStatus != 'ACTIVE' and createProphetStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpStatus != 'ACTIVE' and createDeeparpStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_PENDING
DeepARP:  CREATE_PENDING
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  ACTIVE
CPU times: user 39.4 ms, sys: 0 ns, total: 39.4 ms
Wall time: 5min


In [15]:
%store item_meta_prophet_forecast_arn
%store item_meta_deeparp_forecast_arn
%store ItemMetaForecastProphetExportJobArn
%store ItemMetaForecastDeeparExportJobArn



Stored 'item_meta_prophet_forecast_arn' (str)
Stored 'item_meta_deeparp_forecast_arn' (str)
Stored 'ItemMetaForecastProphetExportJobArn' (str)
Stored 'ItemMetaForecastDeeparExportJobArn' (str)
